# Práctica Imagen - Parte 3

In [ ]:
#!pip install tensorflow
#!pip install opencv-python
#!pip install keras
#!pip install imageAi
#!mkdir input models output

# Custom Object detector

En este caso se ha automatizado el proceso de etiquetado, de imagenes para el _object detector_, el cual consiste en poner un recuadro dentro de la imagen del objeto que se desea catalogar y ponerle una etiqueta-nombre de clase. Puesto que todas las imágenes son del mismo tamaño, se ha podido automatizar el proceso de etiquetado (que al final consiste en tener la información de la etiqueta asignada en un XML). Todo este pre procesado viene explicado a continuación en ingles (he optado por tenerlo en inglés así puedo conservarlo en mi GitHub y tenerlo como muestra de trabajo que he hecho)

Ejemplo de :https://medium.com/deepquestai/train-object-detection-ai-with-6-lines-of-code-6d087063f6ff

Programa para etiquetar: https://medium.com/deepquestai/object-detection-training-preparing-your-custom-dataset-6248679f0d1d



## This will be done for train and validation folders
- First of all decide what fruits you will use (I use less and less pictures due to processor issues) and the number of pictures to get from the dataset (min 200 per class is recomended)
- Then, as images are spread in different directories (for example: Apple Red, Apple Golden) I group them in one sigle directory (Apple) and as inside those directories image filenames are repeated, I rename the files as follows {fruit}{number}
- After that, all fruits used are moved into one single directory and format _.jpg_ is added to filenames


In [12]:
import shutil
import os
import numpy as np    
import pandas as pd
import re



### Train - images/train/images folder (not tagging yet)

I will use only 3 fruits due to processor issues: apples, because they are in almost every test picture that contians multiple fruits; grapes because I want to see if the detector will tag every single grape or only one; bananas (same reason as grapes)

In [15]:
source_dir = 'fruits360/Training/'
target_dir = 'images/train/images/'

#These are all classes of fruits, without considering their subclass (GOLDEN apple)
#classes=(["Apple","Apricot","Avocado","Banana","Beetroot","Cactus","Cantaloupe","Cauliflower","Cherry","Chestnut","Clementine","Cocos","Corn",
#         "Cucumber","Eggplant","Fig","Ginger","Granadilla","Grape","Grapefruit","Guava","Hazelnut","Kaki","Kiwi","Kohlrabi","Kumquats","Lemon","Limes",
#         "Lychee","Mandarine","Mango","Mangostan","Maracuja","Mulberry","Nectarine","Nut","Onion","Orange","Papaya","Passion","Peach","Pear","Pepino",
#         "Pepper","Physalis","Pineapple","Pitahaya","Plum","Pomelo","Potato","Quince","Raspberry","Rambutan","Redcurrant","Salak","Strawberry","Tamarillo","Tangelo",
#         "Tomato","Walnut","Watermelon"]
#        )


fruits_to_be_used=["Apple","Banana","Grape"]
n_samples=200

As I want 'n_samples', and there are N multiple subclasses of the same fruit, I want to do it as precise as I can and split those 'n_samples' into N

Fruits to be used is auto-explicative; n_subsamples is the number of samples that will be taken from each subclass directory (if n_samples = 200 and 
there are 2 classes of Apples, then n_subsamples = 100); 


In [ ]:
## As I want 'n_samples', and there are N multiple subclasses of the same fruit, I want to do it as precise as I can and split those 'n_samples' into N
#Fruits to be used is auto-explicative; n_subsamples is the number of samples that will be taken from each subclass directory (if n_samples = 200 and 
#there are 2 classes of Apples, then n_subsamples = 100); 

fruit_info=pd.DataFrame(columns=['fruits_to_be_used','n_subsamples'])
fruit_info.fruits_to_be_used=fruits_to_be_used

#for 'n_subsample'...
#I substract the subclass of the directories (Apple directory will appear in that list as many times as subclasses there are)
n_subsamples=[]
fruits_subclasses= [x.split(" ")[0] for x in os.listdir(source_dir)]
for i in fruits_to_be_used:
    n_subsamples.append(fruits_subclasses.count(i))
fruit_info.n_subsamples=n_subsamples

fruit_info.head()

Grouping different variants of one fruit in one folder and renaming them so I can automate in next step

In [ ]:
#Grouping fruits by their main class in separeted directories and adding '.jpg' format

#For every listed folder is a class/subclass of fruit that contains images
for i in  np.array(os.listdir(source_dir)):
    try:
        fruit_name=i.split(" ")[0] #First element of the directory name is always the fruit name (APPLE golden)
        if fruit_name in fruits_to_be_used: #If this is the fruit we want...
            fruits = np.array(os.listdir(source_dir+i)) #Now we read all the fruits that are inside the directory
            n_fruits=np.arange(0,fruits.shape[0]) #To know how many fruits there are inside and also to assign a number to new filename
            
            os.mkdir(target_dir+fruit_name) #Creating new folder for grouping all fruits of one class (Apple, for example)
            #If its OK (not exists already - means that first subclass directory is being read)
            
            threshhold=n_samples//fruit_info[fruit_info['fruits_to_be_used']==fruit_name].n_subsamples.item()+1 ## Threshold is n_samples/X +1 
            for index, value in enumerate(fruits): #Para cada una de las frutas dentro de dicho directorio, la movemos al directorio nuevo
                if n_fruits[index]< threshhold:
                    shutil.copyfile(source_dir+i+"/"+value, target_dir+fruit_name+"/"+fruit_name+str(n_fruits[index])+'.jpg') #New name {fruit}{number}
    except FileExistsError:
        #If folder already exists, means that there are already some fruit images inside therefore the number will not be 0
        #Si ya existe la carpeta es que el número que le tengo que asignar a la fruta no empieza en 0
        n_fruits_before= np.array(os.listdir(target_dir+fruit_name+"/")).shape[0]
        n_fruits_new=np.arange(n_fruits_before,n_fruits_before+fruits.shape[0])
        threshhold=n_samples//fruit_info[fruit_info['fruits_to_be_used']==fruit_name].n_subsamples.item()+1 ## Threshold is n_samples/X +1 
        for index, value in enumerate(fruits): #Para cada una de las frutas dentro de dicho directorio, la movemos al directorio nuevo
            if n_fruits[index]< threshhold:
                shutil.copyfile(source_dir+i+"/"+value, target_dir+fruit_name+"/"+fruit_name+str(n_fruits_new[index])+'.jpg') #Le damos un nombre nuevo que es {fruta}{número}
    except NotADirectoryError:
        pass

After all fruits have the right name, we just shuffle them all into one folder for training, no longer splitten in folders per fruit type

In [ ]:
#Now I'd really like to have fruits in images/train folder and not inside different folders like images/train/Apple
source_dir='images/train/images/'
target_dir='images/train/images/'
for i in  np.array(os.listdir(source_dir)):
    try:
        fruits = np.array(os.listdir(source_dir+i)) #Read fruits inside directory
        for index, value in enumerate(fruits): #Every fruit is moved
                shutil.move(source_dir+i+"/"+value, target_dir+value) 
        #After moving all fruits, the old directory is removed -> well, I actually couldnt due to permissions issues and I did not fix it. Do it manually
        #os.remove(source_dir+i)
    except NotADirectoryError:
        pass

In [ ]:
#In case you want to delete some images randomly

#from random import sample 
#files=np.array(os.listdir('images/train/images'))
#num_files=files.shape[0]

#randomlist = random.sample(range(1, num_files), num_files//2)
#for i in randomlist:
#    os.remove('images/train/images/'+files[i])

### Validation - images/validation/images folder
Now folder images/validation folder is going to be set up. We will repeat the same process as before and use the same number of validation samples, in terms of imags manipulation. All code will be the same, but image names will include a suffix *_validation*


In [54]:
source_dir = 'fruits360/Test/'
target_dir = 'images/validation/images/'

#These are all classes of fruits, without considering their subclass (GOLDEN apple)
#classes=(["Apple","Apricot","Avocado","Banana","Beetroot","Cactus","Cantaloupe","Cauliflower","Cherry","Chestnut","Clementine","Cocos","Corn",
#         "Cucumber","Eggplant","Fig","Ginger","Granadilla","Grape","Grapefruit","Guava","Hazelnut","Kaki","Kiwi","Kohlrabi","Kumquats","Lemon","Limes",
#         "Lychee","Mandarine","Mango","Mangostan","Maracuja","Mulberry","Nectarine","Nut","Onion","Orange","Papaya","Passion","Peach","Pear","Pepino",
#         "Pepper","Physalis","Pineapple","Pitahaya","Plum","Pomelo","Potato","Quince","Raspberry","Rambutan","Redcurrant","Salak","Strawberry","Tamarillo","Tangelo",
#         "Tomato","Walnut","Watermelon"]
#        )


fruits_to_be_used=["Apple","Banana","Grape"]
n_samples=200

In [55]:
fruit_info=pd.DataFrame(columns=['fruits_to_be_used','n_subsamples'])
fruit_info.fruits_to_be_used=fruits_to_be_used

#for 'n_subsample'...
#I substract the subclass of the directories (Apple directory will appear in that list as many times as subclasses there are)
n_subsamples=[]
fruits_subclasses= [x.split(" ")[0] for x in os.listdir(source_dir)]
for i in fruits_to_be_used:
    n_subsamples.append(fruits_subclasses.count(i))
fruit_info.n_subsamples=n_subsamples

fruit_info.head()

,fruits_to_be_used,n_subsamples
0,Apple,13
1,Banana,3
2,Grape,6


In [57]:
#Grouping fruits by their main class in separeted directories and adding '.jpg' format

#For every listed folder is a class/subclass of fruit that contains images
for i in  np.array(os.listdir(source_dir)):
    try:
        fruit_name=i.split(" ")[0] #First element of the directory name is always the fruit name (APPLE golden)
        if fruit_name in fruits_to_be_used: #If this is the fruit we want...
            fruits = np.array(os.listdir(source_dir+i)) #Now we read all the fruits that are inside the directory
            n_fruits=np.arange(0,fruits.shape[0]) #To know how many fruits there are inside and also to assign a number to new filename
            
            os.mkdir(target_dir+fruit_name) #Creating new folder for grouping all fruits of one class (Apple, for example)
            #If its OK (not exists already - means that first subclass directory is being read)
            
            threshhold=n_samples//fruit_info[fruit_info['fruits_to_be_used']==fruit_name].n_subsamples.item()+1 ## Threshold is n_samples/X +1 
            for index, value in enumerate(fruits): #Para cada una de las frutas dentro de dicho directorio, la movemos al directorio nuevo
                if n_fruits[index]< threshhold:
                    shutil.copyfile(source_dir+i+"/"+value, target_dir+fruit_name+"/"+fruit_name+'_validation'+str(n_fruits[index])+'.jpg') #New name {fruit}{number}
    except FileExistsError:
        #If folder already exists, means that there are already some fruit images inside therefore the number will not be 0
        #Si ya existe la carpeta es que el número que le tengo que asignar a la fruta no empieza en 0
        n_fruits_before= np.array(os.listdir(target_dir+fruit_name+"/")).shape[0]
        n_fruits_new=np.arange(n_fruits_before,n_fruits_before+fruits.shape[0])
        threshhold=n_samples//fruit_info[fruit_info['fruits_to_be_used']==fruit_name].n_subsamples.item()+1 ## Threshold is n_samples/X +1 
        for index, value in enumerate(fruits): #Para cada una de las frutas dentro de dicho directorio, la movemos al directorio nuevo
            if n_fruits[index]< threshhold:
                shutil.copyfile(source_dir+i+"/"+value, target_dir+fruit_name+"/"+fruit_name+'_validation'+str(n_fruits_new[index])+'.jpg') #Le damos un nombre nuevo que es {fruta}{número}
    except NotADirectoryError:
        pass

In [58]:
#Now I'd really like to have fruits in images/train folder and not inside different folders like images/train/Apple
source_dir='images/validation/images/'
target_dir='images/validation/images/'
for i in  np.array(os.listdir(source_dir)):
    try:
        fruits = np.array(os.listdir(source_dir+i)) #Read fruits inside directory
        for index, value in enumerate(fruits): #Every fruit is moved
                shutil.move(source_dir+i+"/"+value, target_dir+value) 
        #After moving all fruits, the old directory is removed -> well, I actually couldnt due to permissions issues and I did not fix it. Do it manually
        #os.remove(source_dir+i)
    except NotADirectoryError:
        pass

## For automatiting generation of XML from LabelIMG program, that contains tags-labels in each picture + some other info.

I CONSIDER that all images, after reviewing some of them, (they do, if Im not wrong, they are all small squared ones). I manually generate one XML and then I manipulate it to generate more copies. As images are small ones, the tag is the whole image


In [ ]:
# Ahora resulta que quiero automatizar la generación del fichro xml que contiene los datos de la imagen que ha sido taggeada
# Tengo que leer el xml y modificar algunos campos. Por suerte el tamaño de la imagen es el mismo en todos los casos, o al menos lo asumo 

import xml.etree.ElementTree as ET

# Información de la librería: https://docs.python.org/3/library/xml.etree.elementtree.html

In [ ]:
# As an Element, root has a tag and a dictionary of attributes and also has child nodes over which we can iterate:
#Reading the XML that will be used as template
tree = ET.parse('images/train/annotations/Apple0.xml')
root = tree.getroot()

##Python does not show the XML content but it is containted as far as I could see after doing some testing

for child in root:
    print(child.tag,child.attrib)
    
# Me interesa tocar el campo 'filename' para adaptarlo a cada imagen, 'path' y 'object -> name', que dentro tiene el tag especificado


### Train - Automating XML generation - images/train/annotations folder

In order to automate the XML generated by the program that tags the fruits, I manipulate some fields of that XML: filename, path and name (tag)

In [ ]:
# We need the current directory to append to it its suffix (the rest of its path)
import xml.etree.ElementTree as ET
pwd=os.path.abspath(os.getcwd())+"/"

In [ ]:
# XML that is going to be our template
tree = ET.parse('images/train/annotations/Apple0.xml')
root = tree.getroot()


source_dir=pwd+'images/train/images/'
target_dir=pwd+'images/train/annotations/'

#I need to know the fruit name because it is going to be the tag-label
images = np.array(os.listdir(source_dir))

#For every image we will generate a new xml, based on the one we read before because they share the same architecture
for filename in images:
    fruit_number=filename.split(".")[0]
    match = re.match(r"([a-zA-Z]+)", fruit_number, re.I)
    if match:
        fruit_name= match.groups()[0]
    ## Fields that need to be updated:  'filename', 'path', and 'name' (inside 'object' tag)
    #In order to have access to the content of the tag, we use '.text'
    root.find('filename').text=filename
    root.find('path').text=source_dir+filename
    root.find('object').find('name').text=fruit_name
    tree.write(target_dir+fruit_number+'.xml')

In [72]:
#HIDDEN FILES: If you use MAC, more likely you will have those files messing up in the directory, what will cause you an error while executing the model
#Folders are 'images' and dont forget about 'annotations'

#!mv images/train/images/.DS_Store .
#!rm -rf images/train/images/.ipynb_checkpoints/ 

#!rm -r images/train/images/.*
#!rm -r images/train/annotations/.*


### Validation - Automating XML generation - images/validation/annotations folder

Idem

In [61]:
# We need the current directory to append to it its suffix (the rest of its path)
import xml.etree.ElementTree as ET
pwd=os.path.abspath(os.getcwd())+"/"

In [63]:
# XML that is going to be our template
tree = ET.parse('images/validation/annotations/Apple_validation0.xml')
root = tree.getroot()

source_dir=pwd+'images/validation/images/'
target_dir=pwd+'images/validation/annotations/'

#I need to know the fruit name because it is going to be the tag-label
images = np.array(os.listdir(source_dir))

#For every image we will generate a new xml, based on the one we read before because they share the same architecture
for filename in images:
    fruit_number=filename.split(".")[0]
    match = re.match(r"([a-zA-Z]+)", fruit_number, re.I)
    if match:
        fruit_name= match.groups()[0]
    ## Fields that need to be updated:  'filename', 'path', and 'name' (inside 'object' tag)
    #In order to have access to the content of the tag, we use '.text'
    root.find('filename').text=filename
    root.find('path').text=source_dir+filename
    root.find('object').find('name').text=fruit_name
    tree.write(target_dir+fruit_number+'.xml')

In [73]:
#!rm -rf images/validation/images/.*
#!rm -rf images/validation/annotations/.*

### Object detector

Download model from: https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

In [75]:
#Código para revisar que no quedan ficheros ocultos que fastidian el entrenamiento


#import cv2
#import numpy as np
#import os
#source1='images/train/annotations/'
#source2='images/train/images/'

#for i in np.array(os.listdir(source)):
#    print(i)
#    if i[0]=='.':
#        print(i)
    #If not sure, execute this two using images as input. 2nd one fails is a non picture is read
    #img = cv2.imread(source1+i)
    #image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Model is trained only with 15 epochs and batch size of 8

In [76]:
#Nos descargamos un modelo preentrenado: https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

#classes=(["Apple","Apricot","Avocado","Banana","Beetroot","Cactus","Cantaloupe","Cauliflower","Cherry","Chestnut","Clementine","Cocos","Corn",
#         "Cucumber","Eggplant","Fig","Ginger","Granadilla","Grape","Grapefruit","Guava","Hazelnut","Kaki","Kiwi","Kohlrabi","Kumquats","Lemon","Limes",
#         "Lychee","Mandarine","Mango","Mangostan","Maracuja","Mulberry","Nectarine","Nut","Onion","Orange","Papaya","Passion","Peach","Pear","Pepino",
#         "Pepper","Physalis","Pineapple","Pitahaya","Plum","Pomelo","Potato","Quince","Raspberry","Rambutan","Redcurrant","Salak","Strawberry","Tamarillo","Tangelo",
#         "Tomato","Walnut","Watermelon"]
#        )

pwd=os.path.abspath(os.getcwd())+"/"
fruits_to_be_used=["Apple","Banana","Grape"]


#— object_names_array: This is an array of the names of all the objects in your dataset. 
#— batch_size: This is the batch size for the training. Kindly note that the larger the batch size, the better the detection accuracy of the saved models. 
#However, due to memory limits on the Nvidia K80 GPU available on Colab, we have to keep this value as 4. The batch size can be values of 8, 16 and so on.
#— num_experiments: This is the number of times we want the training code to iterate on our custom dataset.
#— train_from_pretrained_model: This is used to leverage transfer learning using the pretrained YOLOv3 model we downloaded earlier.


#Once the training starts,
#ImageAI will generate detection_config.json file in the hololens/json folder. This JSON file will be used during detection of objects in images and videos
#ImageAI will create hololens/models folder which is where all generated models will be saved
#You will see at the log like the sample details below.

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory=pwd+"images")
trainer.setTrainConfig(object_names_array=fruits_to_be_used, batch_size=8, num_experiments=15, train_from_pretrained_model="pretrained-yolov3.h5")

trainer.trainModel()

Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 1.00
Anchor Boxes generated.
Detection configuration saved in  /Users/danipanasik/Desktop/objectDetector/images/json/detection_config.json
Evaluating over 613 samples taken from /Users/danipanasik/Desktop/objectDetector/images/validation
Training over 613 samples  given at /Users/danipanasik/Desktop/objectDetector/images/train
Training on: 	['Apple', 'Banana', 'Grape']
Training with Batch Size:  8
Number of Training Samples:  613
Number of Validation Samples:  613
Number of Experiments:  3
Training with transfer learning from pretrained Model
Epoch 1/3
616/616 [==============================] - 6767s 11s/step - loss: 76.2967 - yolo_layer_9_loss: 15.7747 - yolo_layer_10_loss: 14.6543 - yolo_layer_11_loss: 34.3309 - val_loss: 20.2930 - val_yolo_layer_9_loss: 9.2838 - val_yolo_layer_10_loss: 0.0306 - val_yolo_layer_11_loss: 0.3579
Epoch 2/3
616/616 [==============================] - 8004s 13s/step - l

### Evaluate your models 


To ensure that you pick the best model for your custom detection, ImageAI allows you to evaluate the mAP of all the trained models saved in the images/models folder.

In [77]:
# To ensure that you pick the best model for your custom detection, ImageAI allows you to evaluate the mAP of all the trained models saved 
# in the images/models folder.

#The higher the mAP, the better the detection accuracy of the model.

pwd=os.path.abspath(os.getcwd())+"/"

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="images")
trainer.evaluateModel(model_path="images/models", json_path="images/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

#In the first 4 lines, we import the same training class, created the class instance, set the detection model type and and set the path to our dataset’s directory.
#In the 5th line, we called the .evaluateModel function and specified the parameters below
#— model_path: This is the path to the folder containing our models. It can also be the filepath to a specific model.
#— json_file: This is the path to the detection_config.json file saved during the training.
#— iou_threshold: This is our desired minimum Intersection over Union value for the mAP computation. It can be set to values between 0.0 to 1.0
#— object_threshold: This is our desired minimum class score for the mAP computation. It can be set to values between 0.0 to 1.0.
#— nms_threshold: This is our desired Non-maximum suppression for the mAP computation.



Starting Model evaluation....
Evaluating over 613 samples taken from images/validation
Training over 613 samples  given at images/train
skipping the evaluation of images/models/.ipynb_checkpoints since it's not a .h5 file
Model File:  images/models/detection_model-ex-001--loss-0040.397.h5 

Evaluation samples:  613
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
Apple: 0.6683
Banana: 0.3933
Grape: 0.6278
mAP: 0.5632
Model File:  images/models/detection_model-ex-002--loss-0014.567.h5 

Evaluation samples:  613
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
Apple: 0.6897
Banana: 0.7476
Grape: 0.8618
mAP: 0.7664
Model File:  images/models/detection_model-ex-003--loss-0011.311.h5 

Evaluation samples:  613
Using IoU:  0.5
Using Object Threshold:  0.3
Using Non-Maximum Suppression:  0.5
Apple: 0.4054
Banana: 0.8716
Grape: 0.7687
mAP: 0.6819


[{'model_file': 'images/models/detection_model-ex-001--loss-0040.397.h5',
  'using_iou': 0.5,
  'using_object_threshold': 0.3,
  'using_non_maximum_suppression': 0.5,
  'average_precision': {'Apple': 0.6683198114879976,
   'Banana': 0.3933307729121957,
   'Grape': 0.6278112712750261},
  'evaluation_samples': 613,
  'map': 0.5631539518917398},
 {'model_file': 'images/models/detection_model-ex-002--loss-0014.567.h5',
  'using_iou': 0.5,
  'using_object_threshold': 0.3,
  'using_non_maximum_suppression': 0.5,
  'average_precision': {'Apple': 0.6896510270232175,
   'Banana': 0.7475989740306708,
   'Grape': 0.8618257709960145},
  'evaluation_samples': 613,
  'map': 0.7663585906833009},
 {'model_file': 'images/models/detection_model-ex-003--loss-0011.311.h5',
  'using_iou': 0.5,
  'using_object_threshold': 0.3,
  'using_non_maximum_suppression': 0.5,
  'average_precision': {'Apple': 0.40537118941472233,
   'Banana': 0.871642620762183,
   'Grape': 0.7687076517919416},
  'evaluation_samples': 

#### The best one is the 2nd model


The model has been trained with 3 epochs, therefore, it is not expected to see a good object detector in terms of _detecting_ in the image the fruit in the right spot

Time to check how the model detects the fruits. Same images are used as input + one that contains grapes and another with bananas. Ouput is in the analog output folder: _PARTE3_output_

In [9]:
#Time to detect your images

from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("images/models/detection_model-ex-003--loss-0011.311.h5") 
detector.setJsonPath("images/json/detection_config.json")
detector.loadModel()


- First, mix of fruits. Apple should be detected

In [10]:
input_path="input/apple_apricot_nectarine_peach_peach(flat)_pomegranate_pear_plum.jpg"
output_path = "PARTE3_output/DETECTED_apple_apricot_nectarine_peach_peach(flat)_pomegranate_pear_plum.jpg"
detections = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


Banana  :  80.42610883712769  :  [174, 0, 3135, 2247]
Apple  :  89.63481187820435  :  [328, 237, 3047, 2463]
Grape  :  77.58365273475647  :  [328, 237, 3047, 2463]


Model does not define in the right way the square, therefore we are not 100% sure that the apple detected was a correct detection or not
Model also finds grapes, but there are no.

- Apples only

In [11]:
input_path="input/apples1.jpg"
output_path = "PARTE3_output/DETECTED_apples1.jpg"
detections = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


Banana  :  95.37512063980103  :  [306, 117, 2994, 3840]


Model again finds grapes, but there are no. Its probably that only 3 epochs were not enough for the model to distinguish apple's shape and grape's shape. 

**The fact that all images had same size made the model do not understand that a grape is way smaller than an apple**

- Apples and peaches. Apples should be detected

In [12]:
input_path="input/apples_peaches1.jpg"
output_path = "PARTE3_output/DETECTED_apples_peaches1.jpg"
detections = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


Banana  :  92.95088648796082  :  [72, 45, 3564, 2974]
Apple  :  94.86083984375  :  [160, 262, 4202, 2760]


Idem as before

- Apple and grapes

In [13]:
input_path="input/apple_grape.jpg"
output_path = "PARTE3_output/DETECTED_apple_grape.jpg"
detections = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


Apple  :  95.50875425338745  :  [123, 729, 2943, 3823]
Grape  :  92.77992844581604  :  [123, 729, 2943, 3823]


**The fact that apples have a higher % probability than grapes makes me thing that the fact that all images had the same size made the model do not learn grape's size**

- Mangos and bananas. Bananas should be detected

In [14]:
input_path="input/mangos_bananas(lady_finger).jpg"
output_path = "PARTE3_output/DETECTED_mangos_bananas(lady_finger).jpg"
detections = detector.detectObjectsFromImage(input_image=input_path, output_image_path=output_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


Banana  :  86.75123453140259  :  [81, 72, 3130, 2416]
Apple  :  96.18552327156067  :  [0, 228, 3908, 2219]


**The model did not learn anything about bananas**